In [ ]:
#from dotenv import load_dotenv
import os
import requests
import base64
import csv

In [ ]:
# Cargar el archivo .env
#load_dotenv()

# Credenciales de la aplicación (por ahora se hardcodean en el codigo)
client_id = 'dd5330a3e8a748aa8b74f19336b196f1'
client_secret = 'ee6e6be9304c403fa6072e2f7c5d249c'

# Codificar las credenciales en base64
credentials = f"{client_id}:{client_secret}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

In [15]:
# Obtener access token
def GetAccessToken() :
    token_url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': f'Basic {encoded_credentials}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {'grant_type': 'client_credentials'}

    response = requests.post(token_url, headers=headers, data=data)
    access_token = response.json().get('access_token')
    return access_token

In [16]:
# Se guarda el access token
access_token = GetAccessToken()

In [17]:
track_id = '1z1Hg7Vb0AhHDiEmnDE79l'
track_name = 'All the Time - Don Diablo Remix'
playlist_id = '37i9dQZF1DXcZDD7cfEKhW'
artist_id = '1Xylc3o4UrD53lo9CvFvVg'

In [18]:
def GetTrackInfo(track_id) :
    track_url = f'https://api.spotify.com/v1/tracks/{track_id}'
    headers = {'Authorization': f'Bearer {access_token}'}

    response = requests.get(track_url, headers=headers)
    track_info = response.json()
    return track_info

def GetTracksInfo(track_ids) :
    track_url = f"https://api.spotify.com/v1/tracks?ids={','.join(track_ids)}"
    headers = {'Authorization': f'Bearer {access_token}'}

    response = requests.get(track_url, headers=headers)
    track_info = response.json()
    return track_info

In [19]:
def SearchByName(track_name, limit = 5) :
    url = 'https://api.spotify.com/v1/search'
    params = {
        'q': track_name,
        'type': 'track',
        'limit': limit
    }
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(url, headers=headers, params=params)
    search_results = response.json()
    return search_results

In [20]:
def SearchByYear(year_range = '2010-2020' , limit = 5) :
    track_ids = []
    url = 'https://api.spotify.com/v1/search'
    params = {
        'q': f'year: {year_range}',
        'type': 'track',
        'limit': limit
    }
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(url, headers=headers, params=params)
    search_results = response.json()

    for track in search_results['tracks']['items']:
        track_ids.append(track['id'])

    return track_ids

In [21]:
def GetAudioFeatures(track_id) :
    url = f'https://api.spotify.com/v1/audio-features/{track_id}'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(url, headers=headers)
    audio_features = response.json()
    return audio_features


def GetAudiosFeatures(track_ids) :    
    url = f"https://api.spotify.com/v1/audio-features?ids={','.join(track_ids)}"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(url, headers=headers)
    audio_features = response.json()
    return audio_features

In [22]:
def GetPlaylistTracks(playlist_id) :
    playlist_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    playlist_response = requests.get(playlist_url, headers=headers)
    playlist_tracks = playlist_response.json()

    return playlist_tracks

In [23]:
def GetArtistInfo(artist_id) :
    artist_url = f'https://api.spotify.com/v1/artists/{artist_id}'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    artist_response = requests.get(artist_url, headers=headers)
    artist_data = artist_response.json()
    return artist_data

In [33]:
def JoinTrackInfo(track_info, audio_features) :

    new_data = {
        "track_id": track_info['id'],
        "track_name": track_info['name'],
        "track_artist": track_info['artists'][0]['name'],
        "track_artist_id": track_info['artists'][0]['id'],
        "track_popularity": track_info['popularity'],
        "track_album_id": track_info['album']['id'],
        "track_album_name": track_info['album']['name'],
        "album_release_date": track_info['album']['release_date'],
        "danceability": audio_features['danceability'],
        "energy": audio_features["energy"],
        "key": audio_features["key"],
        "loudness": audio_features["loudness"],
        "mode": audio_features["mode"],
        "speechiness": audio_features["speechiness"],
        "acousticness": audio_features["acousticness"],
        "instrumentalness": audio_features["instrumentalness"],
        "liveness": audio_features["liveness"],
        "valence": audio_features["valence"],
        "tempo": audio_features["tempo"],
        "duration_ms": track_info['duration_ms'],
    }

    return new_data

In [ ]:
# obtenemos tracks ids para distintos periodos - el limite maximo por llamada del search es 50

track_ids = []

track_ids.extend(SearchByYear(year_range='1990-1995', limit=50))
track_ids.extend(SearchByYear(year_range='1996-2000', limit=50))
#track_ids.extend(SearchByYear(year_range='2001-2005', limit=50))
#track_ids.extend(SearchByYear(year_range="2006-2010", limit=50))
#track_ids.extend(SearchByYear(year_range="2011-2015", limit=50))

# eliminamos los duplicados
track_ids = list(dict.fromkeys(track_ids))

print(f'Tenemos {len(track_ids)} nuevos track ids')

Tenemos 94 nuevos track ids


In [ ]:
# para toda la lista track_ids obtenemos la track info y audio features

tracks_info = GetTracksInfo(track_ids)
audio_features = GetAudiosFeatures(track_ids)

# guardamos la informacion en una lista

tracks = []

for i in range(len(track_ids)):
    track = JoinTrackInfo(tracks_info['tracks'][i], audio_features['audio_features'][i])
    tracks.append(track)
    # print(f'{i} - {track}')

In [ ]:
# guardamos la nueva lista en datasets/new_song_list.csv

fields = list(tracks[0].keys())

with open('datasets/new_song_list.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fields)
    writer.writeheader()
    writer.writerows(tracks)